In [1]:
import mlflow
from mlflow.metrics.metric_definitions import _token_count_eval_fn

In [2]:
experiment_name = "Prompt Version"
# mlflow.create_experiment(experiment_name, artifact_location="s3://my-byte/Joel/ByteMLFlow")
experiment = mlflow.set_experiment(experiment_name)

In [11]:
SAS_prompt = """Consider yourself as a medical writer.
    Your job is to summarize the given input text like a medical writer summary.
    The output summary must have 4 sections: Background,efficacy,safety and author conclusion sections.
    Must use the following information for author_name(publication_year): {cite_info}


    Background section:
    Must include the author name and the publication year in the following format: 'author_name(publication_year)'
    Must begin with "author_name(publication_year)" .Strictly not to make any changes in the author_name(publication_year).
    Include essential details like study aim,study design,randomization method,control,blinding,population,diagnosis of interest,diagnostic criteria,age,sex,treatment and exposure,number of participants per treatment,disease condition,dose,dosage form,dose frequency,mode of administration,treatment duration,efficacy endpoints,safety endpoints.
    Only include the mentioned essential details if they are available.
    Include the following details only if present: The total number of subjects that were included in the analysis(with split-ups),subset of subjects that were specifically analyzed for safety-related outcomes or parameters(with split-ups), subset of subjects that were analyzed for efficacy-related outcomes(with split-ups) maintaining their verbatim representation.
    Strictly Important: If any detail is not present/explicitly present, the prompt should ignore it, and there should be no generation of model content, such as 'The study does not specify the country where it was conducted'
    Do not explicitly mention the names of the essential details in the output summary like 'The diagnosis of interest was', 'The study design was'.
    Highly Important: If the aim or study design is not explicitly presented, generate and include them in the background section.
    Must retain the intervention details like treatment regimen with dose in the output summary.
    Strictly important to include the demographics of study participants(including mean/median age,percentage of participants split),treatment regimen,inclusion and exclusion criteria as originally presented in the input text.
    Avoid including information about how the study is conducted or the study procedure.
    Include information about medical procedure.
    Highly Important: Include the detailed information(only if available) about how the randomization process was conducted,the method used to assign patients to different treatment arms, maintaining their verbatim representation.
    Must include the data enclosed with brackets.

    Efficacy section:
    Extract the efficacy results including the results of the efficacy endpoints/effectiveness evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Safety section:
    Extract the safety results including the results of the safety endpoints/safety evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Author conclusion:
    Read the entire input text and find out the author conclusion.
    When presenting the author conclusion, include the phrase 'The authors concluded that' at the beginning.
    Highly Important: If the author conclusion is not present explicitly, generate abstractively based on understanding.

    ***Strictly adhere to the following guidelines:***
    Highly important to make sure not to include the section names in the output summary.
    Highly important to make sure not to include headers, groups and number listing in the output summary.
    Highly important to make sure not to give introduction sentence,conclusion sentence while generating summary.
    Safety results should follow the presentation of efficacy results, and the author conclusion should be presented at the end.
    Highly Important: Reproduce sentences in the paragraph mostly verbatim to maintain language flow.
    Ensure there is no repetition of information between different parts of the output summary.
    Strictly avoid including reference information like references to tables, paragraphs, figures, etc.
    Do not add newline characters in any other part of the output summary.
    Incorporate very concise form of study limitations from the input text into the output summary, only if available. Don't generate on your own.
    Organize the information in chronological order.
    Strictly avoid providing any false information.
    Highly important to differentiate the efficacy,safety and author conclusion sections by adding a newline character('\n\n').
"""


MAS_prompt = """"Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary with in-text citations and the summary should be minimum 1000 words, following the structure provided below:

    Summary Structure:
    The first paragraph should briefly outline the common objective, population range across the studies, study design across all the studies, common intervention with dose and dosage, endpoints measured across all the studies, and studies that do not align with the common intervention of all referenced articles. (Very Important: Objective Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)
    The second paragraph should focus on comparing and contrasting the efficacy results of all primary and secondary endpoints, mentioning the treatment number of subjects, treatment duration, and dosage with citations. If the end results of articles are the same, provide a end result and cite all the articles that fall under that must include efficacy of the medical articles under scrutiny and their percentages.(Very Important: Efficacy Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)
    The third paragraph should concentrate on comparing safety and adverse events across all studies with citation. It's crucial to include patient counts and cases across the studies. (Very Important: safety Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)

    The final paragraph should present the overall conclusion of the findings, along with a risk-benefit analysis. (Very Important: Conclusion Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token) 

    Rule to follow: Ensure that facts are cited by Author (Publication_Year) of the corresponding studies.

    Data of multiple clinical research articles is delimited by triple backticks \n```{whole_data_entity}```\n"""

# Mlflow Logging

In [12]:
with mlflow.start_run(run_name='mas prompt',
                      tags={"mlflow.source.git.commit": 1.1,
                            "solution": 'positive'},
                      log_system_metrics=True):

    mlflow.log_text(MAS_prompt, "prompt.txt")
    mlflow.log_param('prompt', MAS_prompt)
    mlflow.log_metric("prompt_token", _token_count_eval_fn([MAS_prompt]).scores[0])


2024/03/28 07:39:31 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/03/28 07:39:32 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/03/28 07:39:32 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [8]:
runs = mlflow.search_runs(filter_string="tag.mlflow.source.git.commit = '1.1' AND tags.mlflow.runName = 'mas prompt'",
                   experiment_names=[experiment_name])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.prompt_token,params.prompt,params.resume,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName
0,24c4e5c2921a4a3da14b6e408b327b9f,588517342630599120,FINISHED,file:///newvolume/mybyte/Joel/Byte/mlruns/5885...,2024-03-27 13:00:00.964000+00:00,2024-03-28 06:51:00.622000+00:00,368.0,"""Act as a medical review writer and review the...",Working,1.1,ubuntu,/newvolume/mybyte/Joel/venv/lib/python3.10/sit...,LOCAL,mas prompt


In [5]:
runid = runs['run_id'][0]
experimentid = runs['experiment_id'][0]

In [7]:
with mlflow.start_run(run_id = runid,
                      experiment_id = experimentid) as run: 
    artifact_uri = run.info.artifact_uri
    print(mlflow.artifacts.load_text(artifact_uri + "/prompt.txt"))
    mlflow.log_param('resume', 'Working')

"Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary with in-text citations and the summary should be minimum 1000 words, following the structure provided below:

    Summary Structure:
    The first paragraph should briefly outline the common objective, population range across the studies, study design across all the studies, common intervention with dose and dosage, endpoints measured across all the studies, and studies that do not align with the common intervention of all referenced articles. (Very Important: Objective Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)
    The second paragraph should focus on comparing and contrasting the efficacy results of all primary and secondary endpoints, mentioning the treatment number of subjects, treatment duration, and dosage with citations. If the end results of articles are the same, provide a end result and cite all 

In [16]:
import pandas as pd
table = pd.read_json('mlruns/235239485690937724/22a5d20829224badb1c89f6af28055b1/artifacts/prompt_token.json',
                     orient='list')

In [17]:
table

,columns,data
0,article,"[Aussems 1974_LMD004944.pdf, Patients and Meth..."
1,content,"[Bjornberg 1986_LMD052123.pdf, TREATMENT OF TI..."
2,summary,[Clinical Progress Report R14889_11_1971_LMD00...
3,Time taken,[Clinical Progress Report_R14889plus35_1973_LM...
4,prompt_token,[Clinical Research Report R148891NL_1985_LMD04...
5,content_token,"[Den Boer 1980_LMD018448.pdf, TRANSLATION FROM..."
6,summary_token,[Internal Report Project 1355.00_Taplin 1976_L...
7,Total_token,"[Mertens 1976_LMD010427.pdf, A Double-Blind St..."


In [18]:
table['data'][0]

['Aussems 1974_LMD004944.pdf',
 'Patients and Methods \nOne hundred and fifty-seven patients entered this open trial. They all showed skin lesions of apparently fungal origin associated with inflammatory symptoms. \nThe lesions dated from four days to ten years. \nEighty patients out of 157 were known to have been previously treated for their condition before they entered the trial. Above 44 different preparations had been prescribed a total of 126 times to these 80 patients. \nThe results of this previous treatment were recorded in 72 patients as follows : marked improvement but with subsequent relapse in 6 ; moderate improvement in 8 ; minimal improvement in 24 ; no improvement in 34. \nConfirmation of the clinical diagnosis was sought by direct microscopical examination and by culture of skin scrapings from all patients except 15 with eczema marginatum and one with candidosis, fungi being obviously the pathogenic agents. \nSamples were cultured on Sabouraud PS" and PSA mediums at 25